In [4]:
import numpy as np
import pandas as pd
import seaborn as sns



from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score


# model
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
## 확률적 경사하강법을 이용한 선형 분류모델
from sklearn.linear_model import SGDClassifier
## svm 서포트벡터머신 (svc:분류 / svr : 회귀)
from sklearn.svm import SVC

In [5]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [6]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
train.info()

print("------------------------------------")
print("NA 비율")
print("------------------------------------")
p = (train.isna().sum()/len(train)*100).sort_values(ascending=False)
print(p)
print('------------------------------------')
print('duplications 및 other useful info에 대한 고유값')
print('------------------------------------')
u = train.nunique().sort_values()
print(u)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
------------------------------------
NA 비율
------------------------------------
Cabin          77.104377
Age            19.865320
Embarked        0.224467
PassengerId     0.000000
Survived        0.000000
Pclass          0.000000
Name          

#### Missing values:
* Case 1: **'Cabin'** 77% of missing values. 데이터가 3/4이 누락되어 있는 데이터를 모방할 수도 있기 때문에 이 데이터는 삭제한다.

* Case 2: **'Age'** with 20% of missing values. 

* Case 3: **'Embarked'** with 0.2% of missing values. 


#### Categorical values
인코딩하거나 삭제해야하는 범주형 변수

* Case 1: **'Sex'** 값이 2개뿐이면 레이블 인코더를 할 수 있다.

* Case 2: **'Name'** 유용한 정보를 제공하지 않으므로 drop한다.

* Case 3: **'Ticket'** 유용한 정보를 제공하지 않으므로 drop한다.

* Case 4: **'Cabin'** drop by missing 70% of values, 결측치가 너무 많이 나옴.

* Case 5: **'Embarked'** has 3 possible values. 

In [9]:
# Embarked 메뉴얼 repacement 수치 확인
train['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [26]:
def cleanData(data):
    
    #데이터 삭제
    del data(['Cabin','Name','Ticket'])
    
    #Missing valuse case2
    data['Age'] = data.groupby(['Pclass','Sex'])['Age'].transform(lambda x : x.fillna(x.median()))
    
    data['Fare'] = data.groupby(['Pclass','Sex'])['Fare'].transform(lambda x : x.fillna(x.median()))
    
    #Missing valuse case3
    data.dropna(axis=0, subset=['Embarked'], inplace=True)
    
    #라벨 인코딩
    le = preprocessing.LabelEncoder()
    
    #성별
    data['Sex'].replace({'male':0, 'female':1}, inplace=True)
    
    #Embarked
    data['Embarked'].replace({'S':0, 'C':1, 'Q':2}, inplace=True)
    
    return data

SyntaxError: cannot delete function call (<ipython-input-26-b55aa7c60b2a>, line 4)

In [17]:
clean_train = cleanData(train)
clean_test = claenData(test)

KeyError: "['Cabin' 'Name' 'Ticket'] not found in axis"